In [1]:
# Install pyspark
!pip install pyspark networkx nx_altair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 45 kB/s 
     |████████████████████████████████| 199 kB 64.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=65438e292be392545cc14a306225cc86e8a2d047f4120779d68e2dd83eaeba26
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
from functools import reduce

import networkx as nx
import nx_altair as nxa
from pyspark.sql import DataFrame, Row, SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window

# Create a Spark Session
spark = (
    SparkSession.builder.master("local[*]").appName("Spark Session")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12")
    .getOrCreate()
)

from graphframes import GraphFrame

# Check Spark Session Information
spark

In [3]:

loyalty_records = spark.createDataFrame(
    [
        Row(email="alice@readmail.test", loyalty_id=101),
        Row(email="blake@readmail.test", loyalty_id=102),
        Row(email="clare@readmail.test", loyalty_id=103),
        Row(email="dylan@readmail.test", loyalty_id=104),
        Row(email="ethan@readmail.test", loyalty_id=105),
        Row(email="fiona@readmail.test", loyalty_id=106),
        Row(email="grant@readmail.test", loyalty_id=107),
        Row(email="hanna@readmail.test", loyalty_id=108),
    ],
)

ecommerce_records = spark.createDataFrame(
    [
        Row(email="alice@readmail.test", loyalty_id=101),
        Row(email="blake@readmail.test", loyalty_id=102),
        # Same email address, different loyalty_id
        Row(email="ethan@readmail.test", loyalty_id=112),
        # Same email address, different loyalty_id
        Row(email="fiona@readmail.test", loyalty_id=212),
        Row(email="grant@readmail.test", loyalty_id=None),
        Row(email="hanna@readmail.test", loyalty_id=None),
    ],
)

subscriber_records = spark.createDataFrame(
    [
        # Differfent email domain, same loyalty_id
        Row(email="alice@bulkmail.test", loyalty_id=101),
        # Misspelled email username, same loyalty_id
        Row(email="bloke@readmail.test", loyalty_id=102),
        Row(email="dylan@readmail.test", loyalty_id=None),
        # Many loyalty_ids by the same email address
        Row(email="grant@readmail.test", loyalty_id=157),
        Row(email="grant@readmail.test", loyalty_id=257),
        Row(email="grant@readmail.test", loyalty_id=357),
    ],
)


customers = loyalty_records.union(ecommerce_records).union(subscriber_records)

customers

email,loyalty_id
alice@readmail.test,101
blake@readmail.test,102
clare@readmail.test,103
dylan@readmail.test,104
ethan@readmail.test,105
fiona@readmail.test,106
grant@readmail.test,107
hanna@readmail.test,108
alice@readmail.test,101
blake@readmail.test,102


In [4]:
# Vertics DataFrame
vertices = customers.withColumn(
    "id", F.row_number().over(Window.partitionBy(F.lit(1)).orderBy(F.asc("email")))
)

vertices


email,loyalty_id,id
alice@bulkmail.test,101,1
alice@readmail.test,101,2
alice@readmail.test,101,3
blake@readmail.test,102,4
blake@readmail.test,102,5
bloke@readmail.test,102,6
clare@readmail.test,103,7
dylan@readmail.test,104,8
dylan@readmail.test,null,9
ethan@readmail.test,105,10


In [5]:
IDENTIFIER_FIELDS = ["email", "loyalty_id"]

identifier_frames = (
    vertices.select(
        "id",
        F.col(idef).alias("link_value"),
        F.lit(idef).alias("link_type"),
        F.lit("Shares " + idef).alias("action"),
    )
    for idef in IDENTIFIER_FIELDS
)

identifiers = reduce(DataFrame.unionAll, identifier_frames).where(
    F.col("link_value").isNotNull()
)

identifiers

id,link_value,link_type,action
1,alice@bulkmail.test,email,Linked by email
2,alice@readmail.test,email,Linked by email
3,alice@readmail.test,email,Linked by email
4,blake@readmail.test,email,Linked by email
5,blake@readmail.test,email,Linked by email
6,bloke@readmail.test,email,Linked by email
7,clare@readmail.test,email,Linked by email
8,dylan@readmail.test,email,Linked by email
9,dylan@readmail.test,email,Linked by email
10,ethan@readmail.test,email,Linked by email


In [6]:
mirror = identifiers.toDF(*("_" + col for col in identifiers.columns))

mirror

_id,_link_value,_link_type,_action
1,alice@bulkmail.test,email,Linked by email
2,alice@readmail.test,email,Linked by email
3,alice@readmail.test,email,Linked by email
4,blake@readmail.test,email,Linked by email
5,blake@readmail.test,email,Linked by email
6,bloke@readmail.test,email,Linked by email
7,clare@readmail.test,email,Linked by email
8,dylan@readmail.test,email,Linked by email
9,dylan@readmail.test,email,Linked by email
10,ethan@readmail.test,email,Linked by email


In [7]:
join_conditions = [
    (F.col("id") < F.col("_id")) & (F.col("link_value") == F.col("_link_value"))
]

edges = identifiers.join(mirror, join_conditions).select(
    F.col("id").alias("src"),
    F.col("_id").alias("dst"),
    "link_value",
    "link_type",
    "action",
)

edges

src,dst,link_value,link_type,action
2,3,alice@readmail.test,email,Linked by email
4,5,blake@readmail.test,email,Linked by email
8,9,dylan@readmail.test,email,Linked by email
10,11,ethan@readmail.test,email,Linked by email
12,13,fiona@readmail.test,email,Linked by email
14,18,grant@readmail.test,email,Linked by email
14,17,grant@readmail.test,email,Linked by email
14,16,grant@readmail.test,email,Linked by email
14,15,grant@readmail.test,email,Linked by email
15,18,grant@readmail.test,email,Linked by email


In [8]:
# create graph
graph = GraphFrame(vertices, edges)

# run connected components
spark.sparkContext.setCheckpointDir("/tmp/checkpoints")
cc = graph.connectedComponents()

cc

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:149: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  "DataFrame.sql_ctx is an internal property, and will be removed "
/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


email,loyalty_id,id,component
alice@bulkmail.test,101,1,1
alice@readmail.test,101,2,1
alice@readmail.test,101,3,1
blake@readmail.test,102,4,4
blake@readmail.test,102,5,4
bloke@readmail.test,102,6,4
clare@readmail.test,103,7,7
dylan@readmail.test,104,8,8
dylan@readmail.test,null,9,8
ethan@readmail.test,105,10,10


In [9]:
G = nx.Graph()

node_data = (
    cc.withColumn("component", F.col("component").astype("string"))
    .toPandas()
    .set_index("id")
    .to_dict("index")
)

for node_id, data in node_data.items():
    G.add_node(node_id, **data)

edge_data = (
    edges.toPandas()
    .drop_duplicates(subset=["src", "dst"])
    .set_index(["src", "dst"])
    .to_dict("index")
)

for edge_id, data in edge_data.items():
    G.add_edge(edge_id[0], edge_id[1], **data)


nxa.draw_networkx(
    G=G,
    pos=nx.spring_layout(G),
    node_color="component",
    node_tooltip=["component", "email", "loyalty_id"],
    edge_tooltip=["action"],
).properties(
    title="Identity Graph View",
    height=800,
    width=1200,
).interactive()

alt.LayerChart(...)